# Hands Classification
We have recording of hands in 3 different scenarios.  
1. Spontan- two hand moving freely.
2. Sync- two hand moving, trying to be synchronized.
3. Alone- one hand moving freely.  

Our target is to classify hands to one of the categories according to their recorded movements and position. 

In [1]:
#Libraries import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
#constant randoms in different runs
np.random.seed(0)

In [2]:
#understanding the data
#example data:
Evyatar_spon=pd.read_csv('Datasets/Hands/Training/Evyatar Cohen/Evyatar636771052727603804Spontan.csv')

In [3]:
Evyatar_spon.head()

,Time,Frame ID,Hand Type,# hands,Position X,Position Y,Position Z,Velocity X,Velocity Y,Velocity Z,...,Yaw,Wrist Pos X,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength
0,335.9984,128556,right,2,128.63570,224.0294,-12.29588,-18.26243,-140.54450,-117.57790,...,-1.588251,177.5125,173.5658,-14.569630,276.8753,-33.68264,91.61740,0.0,1.250576,0.0
1,335.9984,128556,left,2,-93.24845,143.9423,-37.81597,-109.09820,240.38980,-67.62959,...,0.873458,-147.0455,109.9380,-5.326688,-329.8577,-34.41571,93.88171,0.0,0.014736,0.0
2,336.0150,128558,right,2,129.15550,221.9923,-15.00181,47.36464,-64.25022,-168.85850,...,-1.528183,177.7894,171.1952,-14.677850,276.0645,-33.15913,97.88171,0.0,1.069930,0.0
3,336.0150,128558,left,2,-94.86554,148.5542,-39.00158,-80.99012,278.60220,-66.79356,...,0.879703,-148.2022,113.0909,-7.375025,-326.9279,-39.97873,85.98000,0.0,0.000000,0.0
4,336.0315,128560,right,2,129.87070,220.7944,-18.38681,35.05422,-84.74242,-205.72030,...,-1.461710,178.3730,169.9159,-15.246240,276.1767,-30.72506,104.13920,0.0,0.916969,0.0


In [4]:
#parameters
Evyatar_spon.loc[0].index

Index(['Time', ' Frame ID', ' Hand Type', ' # hands', ' Position X',
       ' Position Y', ' Position Z', ' Velocity X', ' Velocity Y',
       ' Velocity Z', ' Pitch', ' Roll', ' Yaw', ' Wrist Pos X',
       ' Wrist Pos Y', ' Wrist Pos Z', ' Elbow pos X', ' Elbow Pos Y',
       ' Elbow Pos Z', ' Grab Strenth', ' Grab Angle', ' Pinch Strength'],
      dtype='object')

In [5]:
# frames per second
Evyatar_spon[(Evyatar_spon['Time']>336) & (Evyatar_spon['Time']<337)]

,Time,Frame ID,Hand Type,# hands,Position X,Position Y,Position Z,Velocity X,Velocity Y,Velocity Z,...,Yaw,Wrist Pos X,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength
2,336.0150,128558,right,2,129.15550,221.9923,-15.001810,47.364640,-64.25022,-168.85850,...,-1.528183,177.7894,171.19520,-14.677850,276.0645,-33.159130,97.88171,0.0,1.069930,0.0
3,336.0150,128558,left,2,-94.86554,148.5542,-39.001580,-80.990120,278.60220,-66.79356,...,0.879703,-148.2022,113.09090,-7.375025,-326.9279,-39.978730,85.98000,0.0,0.000000,0.0
4,336.0315,128560,right,2,129.87070,220.7944,-18.386810,35.054220,-84.74242,-205.72030,...,-1.461710,178.3730,169.91590,-15.246240,276.1767,-30.725060,104.13920,0.0,0.916969,0.0
5,336.0315,128560,left,2,-95.16259,152.5339,-39.945700,-11.372930,232.80310,-53.32079,...,0.895820,-148.3343,115.92950,-9.437292,-322.0559,-44.423830,81.02193,0.0,0.000000,0.0
6,336.0481,128562,right,2,131.07210,219.2134,-22.055110,36.516350,-153.35370,-212.82590,...,-1.380241,179.5035,168.44820,-15.568560,276.6018,-31.228400,105.96400,0.0,0.798334,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,336.9594,128667,left,2,-76.96600,115.3488,8.684893,73.801640,-145.98930,-228.56920,...,1.000582,-139.3870,94.56170,36.730190,-369.6905,34.979000,123.28030,0.0,1.308134,0.0
118,336.9758,128669,right,2,140.29330,167.0396,-84.622220,-5.801827,241.61630,115.49260,...,-0.478863,174.9576,137.15860,-30.300330,234.9738,-9.023824,167.46080,0.0,0.000000,0.0
119,336.9758,128669,left,2,-75.37846,114.4238,4.754050,105.417400,-52.92049,-245.68910,...,0.981195,-137.1051,92.73209,33.644910,-365.1103,26.101660,121.13610,0.0,1.057895,0.0
120,336.9931,128671,right,2,140.24220,172.7196,-83.057540,15.962420,313.52550,94.47213,...,-0.474993,174.3975,140.33830,-29.841730,234.8062,-16.986300,159.05110,0.0,0.000000,0.0


Every ~1/60 second we have a "picture" of two hands- left and right.  
Each "picture" contains information about the hands placement, velocity and position.  
We will treat each pair of hands in a frame as one instance (single object).  
The "Alone" files will be paired with the file "HandRight.csv".  

## Data Importing

In [6]:
# data import
from os import listdir
folder=('Datasets/Hands/Training/')
Spon, Sync, Alone=list(), list(), list()
for participant in listdir(folder):
    for file in listdir(folder+participant+'/'):
        if file.endswith('Spontan.csv'):
            spon=pd.read_csv(folder+participant+'/'+file)
        elif file.endswith('Sync.csv'):
            sync=pd.read_csv(folder+participant+'/'+file)
        elif file.endswith('Alone.csv'):
            alone=pd.read_csv(folder+participant+'/'+file)
    Spon.append(spon)
    Sync.append(sync)
    Alone.append(alone)
print ('no. of df in Sync: '+str(len(Sync)))

no. of df in Sync: 9


We now have a list for every scenario.  
Test set:

In [7]:
# import test data
folder=('Datasets/Hands/Validation/')
Spon_test, Sync_test, Alone_test=list(), list(), list()
for participant in listdir(folder):
    for file in listdir(folder+participant+'/'):
        if file.endswith('Spontan.csv'):
            spon=pd.read_csv(folder+participant+'/'+file)
        elif file.endswith('Sync.csv'):
            sync=pd.read_csv(folder+participant+'/'+file)
        elif file.endswith('Alone.csv'):
            alone=pd.read_csv(folder+participant+'/'+file)
    Spon_test.append(spon)
    Sync_test.append(sync)
    Alone_test.append(alone)
print ('no. of df in Spon_test: '+str(len(Spon_test)))

no. of df in Spon_test: 3


We pair each "Alone" dataset with given right hand dataset.

In [8]:
right_hand=pd.read_csv("Datasets/Hands/HandRight.csv")
right_hand.head()

,Time,Frame ID,Hand Type,# hands,Position X,Position Y,Position Z,Velocity X,Velocity Y,Velocity Z,...,Yaw,Wrist Pos X,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength
0,128.2883,15478,right,1,15.67167,226.2064,20.26470,34.26665,-195.214700,-35.043020,...,-0.277800,46.88831,207.3548,86.23589,203.8337,49.33507,237.4553,0.0,0.206389,0.0
1,128.3048,15480,right,1,16.24119,222.9389,20.11781,36.16443,-180.093900,-4.052799,...,-0.277763,47.54861,205.6568,86.42250,203.9553,47.88272,238.4482,0.0,0.243050,0.0
2,128.3218,15482,right,1,16.72461,220.9709,19.94972,26.35298,-95.088190,-11.157620,...,-0.278792,48.15030,204.5182,86.35149,204.6231,45.35921,236.8544,0.0,0.281134,0.0
3,128.3384,15484,right,1,17.09016,219.7124,19.67679,19.46804,-61.755180,-21.303780,...,-0.279768,48.64175,203.9989,86.16056,204.9939,44.18692,236.0927,0.0,0.277691,0.0
4,128.3551,15486,right,1,17.46017,219.3652,19.13128,21.94964,-8.355943,-35.081790,...,-0.278629,48.93135,203.3777,85.59017,204.5952,42.68003,235.2896,0.0,0.283008,0.0


## Data Preprocessing

### Search for NaN values, crop first 7 seconds

We will remove the first 7 seconds from each recording (time to get synchronized).

In [9]:
print('Example df length before: '+str(len(Alone[1])))# frames in Alone second dataframe
for scenario in (Spon,Sync,Alone):
    for df in scenario:
        df.drop(range(2*6*7),inplace=True)
print('Example df length after: '+str(len(Alone[1])))

Example df length before: 4575
Example df length after: 4491


In [10]:
# search for NaN values
c=0
for scenario in (Spon,Sync,Alone):
    for df in scenario:
        print('null in df no. '+ str(c)+': '+str(df.isna().sum().sum()))
        c+=1

null in df no. 0: 0
null in df no. 1: 0
null in df no. 2: 0
null in df no. 3: 0
null in df no. 4: 0
null in df no. 5: 0
null in df no. 6: 0
null in df no. 7: 0
null in df no. 8: 0
null in df no. 9: 0
null in df no. 10: 0
null in df no. 11: 0
null in df no. 12: 0
null in df no. 13: 0
null in df no. 14: 0
null in df no. 15: 0
null in df no. 16: 0
null in df no. 17: 0
null in df no. 18: 0
null in df no. 19: 0
null in df no. 20: 0
null in df no. 21: 0
null in df no. 22: 0
null in df no. 23: 0
null in df no. 24: 0
null in df no. 25: 0
null in df no. 26: 0


We'll reapet the process for the test data and right hand data

In [11]:
# test data
print('length before reduce: '+str(len(Spon_test[0])))
# remove first 7 seconds and count NaN
c=0
for scenario in (Spon_test,Sync_test,Alone_test):
    for df in scenario:
        df.drop(range(2*6*7),inplace=True)
        print('null in df no. '+ str(c)+': '+str(df.isna().sum().sum()))
        c+=1
print('length after reduce: '+str(len(Spon_test[0])))

length before reduce: 8969
null in df no. 0: 0
null in df no. 1: 0
null in df no. 2: 0
null in df no. 3: 0
null in df no. 4: 0
null in df no. 5: 0
null in df no. 6: 0
null in df no. 7: 0
null in df no. 8: 0
length after reduce: 8885


In [12]:
# right hand data
print('length before reduce: '+str(len(right_hand)))
right_hand.drop(range(2*6*7),inplace=True)
print('null in right_hand: '+str(right_hand.isna().sum().sum()))
print('length after reduce: '+str(len(right_hand)))

length before reduce: 4812
null in right_hand: 2304
length after reduce: 4728


We have NaN values in right_hand, we need to take care of them.

In [13]:
# locate NaN values
right_hand.isna().sum()

Time                 0
 Frame ID            0
 Hand Type           0
 # hands             0
 Position X          0
 Position Y          0
 Position Z          0
 Velocity X          0
 Velocity Y          0
 Velocity Z          0
 Pitch               0
 Roll                0
 Yaw                 0
 Wrist Pos X       384
 Wrist Pos Y       384
 Wrist Pos Z       384
 Elbow pos X       384
 Elbow Pos Y       384
 Elbow Pos Z       384
 Grab Strenth        0
 Grab Angle          0
 Pinch Strength      0
dtype: int64

If the NaN values are in the same frame, we can gust drop them, losing 384 frames.  
Otherwise we will need more data.

In [14]:
print('right_hand shape with NaN: '+str(right_hand.shape))
print('right_hand shape without NaN: '+str(right_hand.dropna().shape))
print ('difference: '+str(right_hand.shape[0]-right_hand.dropna().shape[0]))

right_hand shape with NaN: (4728, 22)
right_hand shape without NaN: (4344, 22)
difference: 384


Luckily, we only have only 384 frames with missing data, we will discard them.

In [15]:
right_hand=right_hand.dropna()
right_hand.shape

(4344, 22)

### Reorganizing the data frames
We want to pair together left and right hand from the same frame.

### 2 hands scenarios demo
We split the dataset and merge it horizontally

In [16]:
# the dataframe we will demonstrate on
demo=sync.copy()
demo.head()

,Time,Frame ID,Hand Type,# hands,Position X,Position Y,Position Z,Velocity X,Velocity Y,Velocity Z,...,Yaw,Wrist Pos X,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength
84,925.3400,60509,left,2,-121.3679,284.9276,71.08919,-37.52761,78.14517,32.94556,...,1.521496,-167.1657,227.7851,67.58211,-335.6783,38.53807,130.78630,0.0,0.328604,0.0
85,925.3400,60509,right,2,142.1444,271.0127,74.83476,-36.38242,-63.54549,54.77497,...,-1.401698,192.8051,217.4485,75.04996,373.2006,28.74548,82.42117,0.0,0.027603,0.0
86,925.3743,60511,left,2,-122.7052,287.0048,72.01142,-49.97677,69.69836,35.03852,...,1.522061,-168.4623,229.8258,68.45575,-334.5794,38.19341,130.80850,0.0,0.332486,0.0
87,925.3743,60511,right,2,141.2431,269.0698,76.03043,-35.50513,-88.49073,44.93065,...,-1.403901,191.8715,215.4569,76.01191,377.1630,31.49922,81.69724,0.0,0.045244,0.0
88,925.4076,60513,left,2,-124.1146,288.3193,72.90015,-46.41800,35.69813,28.18376,...,1.511062,-170.4284,231.5497,69.74766,-338.1897,41.28901,131.89810,0.0,0.360908,0.0


In [17]:
demo[' Hand Type'].value_counts()

right    2204
left     2158
Name:  Hand Type, dtype: int64

The data isn't complete. We will correct that when merging the right and left data- we will only include frames which have both hands.

In [18]:
#split dataset
demo_left=demo[demo[' Hand Type']=='left']
demo_right=demo[demo[' Hand Type']=='right']
demo_left.head()

,Time,Frame ID,Hand Type,# hands,Position X,Position Y,Position Z,Velocity X,Velocity Y,Velocity Z,...,Yaw,Wrist Pos X,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength
84,925.3400,60509,left,2,-121.3679,284.9276,71.08919,-37.52761,78.14517,32.945560,...,1.521496,-167.1657,227.7851,67.58211,-335.6783,38.53807,130.7863,0.0,0.328604,0.0
86,925.3743,60511,left,2,-122.7052,287.0048,72.01142,-49.97677,69.69836,35.038520,...,1.522061,-168.4623,229.8258,68.45575,-334.5794,38.19341,130.8085,0.0,0.332486,0.0
88,925.4076,60513,left,2,-124.1146,288.3193,72.90015,-46.41800,35.69813,28.183760,...,1.511062,-170.4284,231.5497,69.74766,-338.1897,41.28901,131.8981,0.0,0.360908,0.0
90,925.4398,60516,left,2,-125.3973,287.9051,73.72894,-14.55321,-53.31699,12.755390,...,1.495221,-172.7207,231.9361,71.27187,-352.0691,53.22682,135.3253,0.0,0.382371,0.0
92,925.4729,60518,left,2,-125.1278,284.3668,73.64545,13.44731,-139.13820,-5.644237,...,1.492570,-173.1618,229.0355,71.46091,-360.6429,60.20063,138.9244,0.0,0.389056,0.0


In [19]:
# merge horizontally, only hands in the same frame (inner)
demo2=pd.merge(demo_left,demo_right,on=['Time',' Frame ID',' # hands'],how='inner')
demo2

,Time,Frame ID,Hand Type_x,# hands,Position X_x,Position Y_x,Position Z_x,Velocity X_x,Velocity Y_x,Velocity Z_x,...,Yaw_y,Wrist Pos X_y,Wrist Pos Y_y,Wrist Pos Z_y,Elbow pos X_y,Elbow Pos Y_y,Elbow Pos Z_y,Grab Strenth_y,Grab Angle_y,Pinch Strength_y
0,925.3400,60509,left,2,-121.36790,284.9276,71.08919,-37.52761,78.14517,32.945560,...,-1.401698,192.8051,217.4485,75.04996,373.2006,28.74548,82.42117,0.0,0.027603,0.0
1,925.3743,60511,left,2,-122.70520,287.0048,72.01142,-49.97677,69.69836,35.038520,...,-1.403901,191.8715,215.4569,76.01191,377.1630,31.49922,81.69724,0.0,0.045244,0.0
2,925.4076,60513,left,2,-124.11460,288.3193,72.90015,-46.41800,35.69813,28.183760,...,-1.403654,190.4933,211.9839,76.95097,379.3175,31.64314,82.27242,0.0,0.077169,0.0
3,925.4398,60516,left,2,-125.39730,287.9051,73.72894,-14.55321,-53.31699,12.755390,...,-1.405593,190.5499,209.6013,78.05990,383.9880,34.35482,86.77981,0.0,0.118110,0.0
4,925.4729,60518,left,2,-125.12780,284.3668,73.64545,13.44731,-139.13820,-5.644237,...,-1.404291,189.7028,206.5573,78.37885,386.9746,35.72657,88.69302,0.0,0.124140,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2153,998.7513,66216,left,2,-98.32814,314.4973,75.12682,41.16818,115.85070,27.614590,...,-1.447439,156.7337,255.0919,101.90250,341.0338,65.01146,125.50060,0.0,0.000000,0.0
2154,998.7836,66219,left,2,-97.31071,319.5955,76.22157,31.36302,143.96100,30.502050,...,-1.447845,157.5847,259.1192,102.11500,336.8353,64.01215,123.47900,0.0,0.000000,0.0
2155,998.8158,66221,left,2,-96.74767,323.8981,76.98316,27.14988,185.62180,33.741870,...,-1.436827,159.1932,263.2726,102.53110,332.9102,62.99343,121.62640,0.0,0.000000,0.0
2156,998.8495,66224,left,2,-95.67720,329.8123,78.13316,47.37696,118.31670,24.152470,...,-1.418975,163.0329,269.7217,103.07490,340.7914,72.96347,121.57070,0.0,0.024162,0.0


We have the data of both hands under the same frame.  
Data with "_x" describes the left hand and data with "_y" describes the right hand.

In [20]:
print('demo2 length: '+str(demo2[' # hands'].size))
print('demo2 [# hands] value count: \n'+str(demo2[' # hands'].value_counts()))

demo2 length: 2158
demo2 [# hands] value count: 
2    2158
Name:  # hands, dtype: int64


We removed all single hand data.

## Alone scenarios
This method will work for 2 hands data, in "Alone" datasets we need to pair the "right_hand" dataset.  
However, if we just merged every "Alone" with "right_hand" the machine will be able to classify them easily.  
To avoid such problem, we will split "right_hand" to 2 subsets, train data and test data (with 1 sec distance between to avoid data leak), we will create a random right_hand dataset to accompany each Alone dataset. The right_hand dataset will have the same length as the corresponding Alone dataset, and will be created randomly from the relevant data (train or test data).  
We will achive that by creating lists of random indexes for every Alone dataset and create the right_hand dataset according to them.

In [21]:
# split "right_hand"
right_hand_train_data=right_hand.iloc[:3000]
# 1 sec interval
right_hand_test_data=right_hand.iloc[3060:]
right_hand_train_data.head()

,Time,Frame ID,Hand Type,# hands,Position X,Position Y,Position Z,Velocity X,Velocity Y,Velocity Z,...,Yaw,Wrist Pos X,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength
84,129.6882,15640,right,1,27.13876,207.4796,52.11142,41.588500,-76.60230,-101.7598,...,-0.265757,58.90767,202.2814,120.3888,236.1884,84.59581,286.1831,0.0,0.585902,0.0
85,129.7048,15642,right,1,27.54398,206.3630,49.93446,22.403330,-47.71859,-129.2814,...,-0.264195,59.17055,201.5768,118.2975,235.2411,83.32476,285.0026,0.0,0.557712,0.0
86,129.7208,15644,right,1,27.66292,206.4216,47.49265,0.895973,15.64516,-146.0183,...,-0.264369,59.23776,201.8046,115.9001,234.3763,83.89951,283.8558,0.0,0.546594,0.0
87,129.7375,15646,right,1,27.41630,207.3747,44.52312,-18.092530,54.98383,-178.6693,...,-0.265373,59.01835,202.4811,112.9170,233.1135,84.00522,281.5832,0.0,0.517014,0.0
88,129.7542,15648,right,1,26.98242,209.4798,41.54232,-26.689180,147.61150,-168.3345,...,-0.265294,58.52890,204.3822,109.9002,231.3450,81.18188,276.5166,0.0,0.467832,0.0


In [22]:
#create demo right hand dataframe
demo=pd.DataFrame()
recipe=np.random.randint(3000, size=len(Alone[3]))# create recipe for Alone[3]
for index in recipe:
    demo=demo.append(right_hand_train_data.iloc[index].copy())#create line for each left hand line
demo

,# hands,Elbow Pos Y,Elbow Pos Z,Elbow pos X,Frame ID,Grab Angle,Grab Strenth,Hand Type,Pinch Strength,Pitch,...,Position Z,Roll,Velocity X,Velocity Y,Velocity Z,Wrist Pos X,Wrist Pos Y,Wrist Pos Z,Yaw,Time
2816,1.0,22.003740,322.1707,181.958700,20849.0,0.998335,0.0,right,0.000000,0.451632,...,87.972540,-0.022126,77.28802,-7.132415,-308.451000,13.606850,157.56890,155.96540,-0.042926,174.8730
2691,1.0,-1.703353,249.9926,145.013800,20609.0,0.000000,0.0,right,0.000000,0.823625,...,0.204449,-0.034195,128.75170,-512.891600,-304.788500,31.844500,151.56740,55.21538,-0.116832,172.7899
1737,1.0,52.186940,303.1174,153.942300,18738.0,2.880009,1.0,right,0.615545,0.804581,...,71.898120,0.089243,-107.20090,180.616300,74.145390,68.462240,243.89270,126.50950,-0.254537,156.5618
919,1.0,89.407630,361.5245,108.309300,17231.0,3.141593,1.0,right,1.000000,0.462263,...,39.191720,0.123790,-132.94850,260.496600,-40.933380,12.545120,127.69400,109.73720,-0.090340,143.4827
847,1.0,20.220150,336.0565,154.969100,17106.0,2.587753,0.0,right,0.737045,0.670885,...,60.431630,0.251412,-52.33714,-292.352700,-3.553731,69.947100,168.43510,124.30890,0.241368,142.4000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1025,1.0,41.260890,286.4287,128.080700,17433.0,0.111760,0.0,right,0.000000,0.505462,...,45.031520,-0.750595,-82.56967,396.170400,42.269700,26.943930,222.88080,109.37190,0.018528,145.2500
330,1.0,173.556700,229.3383,203.243500,16112.0,0.319363,0.0,right,0.000000,0.310827,...,6.256772,-0.022093,-40.31133,467.572000,106.362300,33.288750,306.94840,65.71410,-0.504606,133.7864
2519,1.0,-48.728000,285.4431,156.628100,20240.0,0.585126,0.0,right,0.000000,1.381479,...,157.148800,-0.065906,-106.63970,401.949400,-127.145300,9.360494,154.35570,178.04090,-0.437379,169.5907
3002,1.0,96.912740,341.7314,0.214948,21207.0,2.639150,0.0,right,0.021413,0.220966,...,2.812839,-2.680968,-22.85344,-117.348900,-22.146500,12.407840,92.34872,69.41825,0.414427,177.9711


Lastly, we will switch the 'Time' and 'Frame ID' columns to help with merging.  

In [23]:
#process explanation:
#drop 'Time' and Frame ID' from right hand data while reseting the index to avoid duplicates
dropped=demo.reset_index().drop(columns=['Time', ' Frame ID', 'index'])
dropped.head()

,# hands,Elbow Pos Y,Elbow Pos Z,Elbow pos X,Grab Angle,Grab Strenth,Hand Type,Pinch Strength,Pitch,Position X,Position Y,Position Z,Roll,Velocity X,Velocity Y,Velocity Z,Wrist Pos X,Wrist Pos Y,Wrist Pos Z,Yaw
0,1.0,22.003740,322.1707,181.9587,0.998335,0.0,right,0.000000,0.451632,-3.842576,188.1969,87.972540,-0.022126,77.28802,-7.132415,-308.451000,13.60685,157.5689,155.96540,-0.042926
1,1.0,-1.703353,249.9926,145.0138,0.000000,0.0,right,0.000000,0.823625,14.252420,202.4830,0.204449,-0.034195,128.75170,-512.891600,-304.788500,31.84450,151.5674,55.21538,-0.116832
2,1.0,52.186940,303.1174,153.9423,2.880009,1.0,right,0.615545,0.804581,46.002970,293.9568,71.898120,0.089243,-107.20090,180.616300,74.145390,68.46224,243.8927,126.50950,-0.254537
3,1.0,89.407630,361.5245,108.3093,3.141593,1.0,right,1.000000,0.462263,-3.794198,153.4998,39.191720,0.123790,-132.94850,260.496600,-40.933380,12.54512,127.6940,109.73720,-0.090340
4,1.0,20.220150,336.0565,154.9691,2.587753,0.0,right,0.737045,0.670885,72.368970,210.8473,60.431630,0.251412,-52.33714,-292.352700,-3.553731,69.94710,168.4351,124.30890,0.241368


In [24]:
#extracting time and 'Frame ID' from the corresponding Alone data
time_ID=Alone[3][['Time', ' Frame ID']].copy()
#resetting index for concatenating with 'dropped'
time_ID=time_ID.reset_index().drop(columns='index')
time_ID.head()

,Time,Frame ID
0,290.1505,34023
1,290.1664,34024
2,290.1835,34026
3,290.1996,34028
4,290.2164,34030


In [25]:
# our right hand dataset
pd.concat([time_ID, dropped], axis=1)

,Time,Frame ID,# hands,Elbow Pos Y,Elbow Pos Z,Elbow pos X,Grab Angle,Grab Strenth,Hand Type,Pinch Strength,...,Position Y,Position Z,Roll,Velocity X,Velocity Y,Velocity Z,Wrist Pos X,Wrist Pos Y,Wrist Pos Z,Yaw
0,290.1505,34023,1.0,22.003740,322.1707,181.958700,0.998335,0.0,right,0.000000,...,188.1969,87.972540,-0.022126,77.28802,-7.132415,-308.451000,13.606850,157.56890,155.96540,-0.042926
1,290.1664,34024,1.0,-1.703353,249.9926,145.013800,0.000000,0.0,right,0.000000,...,202.4830,0.204449,-0.034195,128.75170,-512.891600,-304.788500,31.844500,151.56740,55.21538,-0.116832
2,290.1835,34026,1.0,52.186940,303.1174,153.942300,2.880009,1.0,right,0.615545,...,293.9568,71.898120,0.089243,-107.20090,180.616300,74.145390,68.462240,243.89270,126.50950,-0.254537
3,290.1996,34028,1.0,89.407630,361.5245,108.309300,3.141593,1.0,right,1.000000,...,153.4998,39.191720,0.123790,-132.94850,260.496600,-40.933380,12.545120,127.69400,109.73720,-0.090340
4,290.2164,34030,1.0,20.220150,336.0565,154.969100,2.587753,0.0,right,0.737045,...,210.8473,60.431630,0.251412,-52.33714,-292.352700,-3.553731,69.947100,168.43510,124.30890,0.241368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4438,363.6728,42486,1.0,41.260890,286.4287,128.080700,0.111760,0.0,right,0.000000,...,263.6024,45.031520,-0.750595,-82.56967,396.170400,42.269700,26.943930,222.88080,109.37190,0.018528
4439,363.6898,42488,1.0,173.556700,229.3383,203.243500,0.319363,0.0,right,0.000000,...,324.9749,6.256772,-0.022093,-40.31133,467.572000,106.362300,33.288750,306.94840,65.71410,-0.504606
4440,363.7059,42489,1.0,-48.728000,285.4431,156.628100,0.585126,0.0,right,0.000000,...,225.8339,157.148800,-0.065906,-106.63970,401.949400,-127.145300,9.360494,154.35570,178.04090,-0.437379
4441,363.7227,42491,1.0,96.912740,341.7314,0.214948,2.639150,0.0,right,0.021413,...,112.3485,2.812839,-2.680968,-22.85344,-117.348900,-22.146500,12.407840,92.34872,69.41825,0.414427


We will pair those datasets with their Alone corresponding datasets to get 2-hands datasets.

In [26]:
# paired dataset example
right=pd.concat([time_ID, dropped], axis=1)
Alone[3].append(right,ignore_index=True)

,Time,Frame ID,Hand Type,# hands,Position X,Position Y,Position Z,Velocity X,Velocity Y,Velocity Z,...,Yaw,Wrist Pos X,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength
0,290.1505,34023,left,1.0,-33.07419,254.2497,23.241940,54.21996,-577.08650,-131.15690,...,1.576030,-89.653980,215.94370,16.05349,-264.564500,48.10490,-10.420780,0.0,0.564059,0.799974
1,290.1664,34024,left,1.0,-32.08050,248.4454,21.861620,114.74510,-670.24350,-159.39010,...,1.568583,-89.060520,210.61260,15.18296,-268.273900,46.98879,-8.781100,0.0,0.570904,0.755616
2,290.1835,34026,left,1.0,-30.47708,239.2094,19.146540,108.77610,-519.50810,-154.31970,...,1.551061,-88.469710,202.71480,13.50917,-276.099900,48.26530,-6.609064,0.0,0.549327,0.696177
3,290.1996,34028,left,1.0,-28.62491,231.8327,16.163330,83.55526,-337.71690,-161.35970,...,1.537723,-87.227700,196.09360,11.31284,-286.516500,56.57439,-5.504764,0.0,0.547002,0.760822
4,290.2164,34030,left,1.0,-27.14336,222.6850,13.447050,132.96110,-534.63560,-160.92060,...,1.529848,-85.900710,187.03040,9.34878,-295.193100,62.35886,-1.522220,0.0,0.662062,0.790702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8881,363.6728,42486,right,1.0,15.10360,263.6024,45.031520,-82.56967,396.17040,42.26970,...,0.018528,26.943930,222.88080,109.37190,128.080700,41.26089,286.428700,0.0,0.111760,0.000000
8882,363.6898,42488,right,1.0,-10.84456,324.9749,6.256772,-40.31133,467.57200,106.36230,...,-0.504606,33.288750,306.94840,65.71410,203.243500,173.55670,229.338300,0.0,0.319363,0.000000
8883,363.7059,42489,right,1.0,-10.27316,225.8339,157.148800,-106.63970,401.94940,-127.14530,...,-0.437379,9.360494,154.35570,178.04090,156.628100,-48.72800,285.443100,0.0,0.585126,0.000000
8884,363.7227,42491,right,1.0,43.06517,112.3485,2.812839,-22.85344,-117.34890,-22.14650,...,0.414427,12.407840,92.34872,69.41825,0.214948,96.91274,341.731400,0.0,2.639150,0.021413


### Repeating the process on all data frames

#### Alone scenarios
we will add the right hand data as shown in the example above.

In [27]:
#create right hand datasets with the same length, Time and Frame ID
def create_right_hand_for_left (right_hand_database, left_hand_dataframe):
    "create right hand datasets with the same length, Time and Frame ID as the left hand dataset"
    df=pd.DataFrame()
    recipe=np.random.randint(len(right_hand_database), size=len(left_hand_dataframe))# create recipe for left_hand_dataframe
    for index in recipe:
        df=df.append(right_hand_database.iloc[index].copy())# add lines
    # reset index
    df=df.reset_index().drop(columns=['Time', ' Frame ID', 'index'])
    # 'Time' and 'Frame ID' of the left hand
    time_ID=left_hand_dataframe[['Time', ' Frame ID']].copy()
    time_ID=time_ID.reset_index().drop(columns='index')# reset index
    # combine data
    df=pd.concat([time_ID, df], axis=1)
    return df

In [28]:
# create lists of right hand data
print('example df shape beafore: '+str(Alone[4].shape))
right_hand_train=[create_right_hand_for_left(right_hand_train_data,left_dataset) 
                  for left_dataset in Alone]
# test data
right_hand_test=[create_right_hand_for_left(right_hand_test_data,left_dataset) 
                  for left_dataset in Alone_test]
print('no. of right hand training datasets: '+str(len(right_hand_train)))
print('no. of right hand test datasets: '+str(len(right_hand_test)))
print('example dataset:')
right_hand_train[3]
print('example df shape beafore: '+str(Alone[4].shape))


example df shape beafore: (4847, 22)
no. of right hand training datasets: 9
no. of right hand test datasets: 3
example dataset:
example df shape beafore: (4847, 22)


In [29]:
#combining Alone and right_hand data
print('example df shape beafore: '+str(Alone[4].shape))
Alone=[Alone[i].append(right_hand_train[i],ignore_index=True) for i in range(len(Alone))]
Alone_test=[Alone_test[i].append(right_hand_test[i],ignore_index=True) for i in range(len(Alone_test))]
print('example df shape after: '+str(Alone[4].shape))

example df shape beafore: (4847, 22)
example df shape after: (9694, 22)


#### 2 hands scenarios

In [30]:
#function that does the "2 hands scenarios" steps
def left_right_rehape (df):
    "pair together left and right hand from the same frame."
    left=df[df[' Hand Type']=='left'] # split
    right=df[df[' Hand Type']=='right']
    df=pd.merge(left,right,on=['Time',' Frame ID',' # hands'],how='inner') # merge horizontally
    return df 

In [31]:
# train data
print('Example df shape before: '+str(Sync[1].shape))# frames in Alone second dataframe
Spon = [left_right_rehape(df) for df in Spon]
Sync = [left_right_rehape(df) for df in Sync]
Alone = [left_right_rehape(df) for df in Alone]
print('Example df shape after: '+str(Sync[1].shape))

Example df shape before: (8970, 22)
Example df shape after: (4485, 41)


In [32]:
# test data
print('Example df shape before: '+str(Sync_test[1].shape))# frames in Sync_test second dataframe
Spon_test = [left_right_rehape(df) for df in Spon_test]
Sync_test = [left_right_rehape(df) for df in Sync_test]
Alone_test = [left_right_rehape(df) for df in Alone_test]
print('Example df shape after: '+str(Sync_test[1].shape))

Example df shape before: (8881, 22)
Example df shape after: (4396, 41)


In [33]:
# process validation
nulls=0
for scenario in (Spon,Sync,Alone):
    for df in scenario:
        nulls+=df.isna().sum().sum()
print ('NaN in train: '+str(nulls))
nulls=0
for scenario in (Spon_test,Sync_test,Alone_test):
    for df in scenario:
        nulls+=df.isna().sum().sum()
print ('NaN in test: '+str(nulls))

NaN in train: 0
NaN in test: 0
